In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, log_loss, roc_auc_score, confusion_matrix
# импортируйте необходимые вам модули

# заведите словарь со всеми метриками
metrics = {}

# посчитайте метрики из модуля sklearn.metrics
# err_1 — ошибка первого рода
# err_2 — ошибка второго рода
_, err1, _, err2 = confusion_matrix(y_test, prediction, normalize='all').ravel() # ваш код здесь #
auc = roc_auc_score(y_test, probas)# ваш код здесь #
precision = precision_score(y_test, prediction)# ваш код здесь #
recall = recall_score(y_test, prediction)# ваш код здесь #
f1 = f1_score(y_test, prediction)# ваш код здесь #
logloss = log_loss(y_test, prediction)# ваш код здесь #

# запишите значения метрик в словарь
metrics["err1"] = err1
metrics["err2"] = err2
metrics["auc"] = auc
metrics["precision"] = precision
metrics["recall"] = recall
metrics["f1"] = f1
metrics["logloss"] = logloss

In [ ]:

import pandas as pd
import numpy as np
import mlflow
import os
from dotenv import load_dotenv

load_dotenv()

EXPERIMENT_NAME = "churn_marselkamilov"# ваш код здесь (напишите своё уникальное имя эксперимента)
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_marselkamilov"

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# ваш код здесь
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 

pip_requirements = '../requirements.txt' # ваш код здесь
signature = mlflow.models.infer_signature(X_test, prediction) # ваш код здесь
input_example = X_test[:10] # ваш код здесь
metadata = {'model_type': 'monthly'} # ваш код здесь


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    model_info = mlflow.catboost.log_model(cb_model=model,
        artifact_path="models",
        signature=signature,
        metadata=metadata,
        input_example=input_example,
        registered_model_name=REGISTRY_MODEL_NAME,
        pip_requirements=pip_requirements,
        await_registration_for=60
		)

In [ ]:
loaded_model = mlflow.catboost.load_model(model_uri=model_info.model_uri) 
model_predictions = loaded_model.predict(X_test)

assert model_predictions.dtype == int

print(model_predictions[:10])